In [47]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import tabula
from datetime import datetime


In [29]:
url = 'https://coronavirus.ceara.gov.br/boletins/'

html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')

link = soup.find_all('div', 'et_pb_button_0_wrapper')[0].find('a').attrs['href']

filename = Path('covid19_ce.pdf')
response = requests.get(link)
filename.write_bytes(response.content)

283956

In [62]:
tabula.convert_into("covid19_ce.pdf", "covid19_ce.csv", output_format="csv", pages='all')

df = pd.read_csv('covid19_ce.csv')


In [63]:
date = link.split('/')[-1].split('_')

date = date[3] + '/' + date[4] + '/' + date[5]+'20'

date = datetime.strptime(date, '%d/%m/%Y').date()

date

datetime.date(2020, 3, 22)

In [64]:
df = df.T.reset_index(drop=True).T
df = df.drop([0,1])
df = df.iloc[ :,1:]

df.drop([3,4], axis=1, inplace=True)

df.dropna(inplace=True)
df = df.rename(columns={1:'MUNICIO', 2:'CONFIRMADOR', 5:'OBITOS'})

df['DATETIME'] = date

df.to_csv('covid19_ce.csv')